# Sistemi preporuka

Postoji nekoliko različitih kategorija sistema preporučivanja:

* Sistemi zasnovani na popularnosti (engl. *popularity based*) preporučuju uvek najpopularnije entitete. Na primer, to mogu biti pesme sa najviše pregleda na YouTube-u ili restorani sa najvišom ocenom.

* Sistemi zasnovani na sadržaju (engl. *content-based*) vrše preporuke na osnovu karakteristika samih entiteta. Pretpostavka je da će se proizvod sličan proizvodu koji je korisnik već kupio/pregledao/odabrao takođe dopasti. 
<img src='assets/content_based.png'>

* Sistemi zasnovani na uzajamnom filtriranju (engl. *collaborative filtering*) vrše preporuke na osnovu znanja o ponašanju grupe. Pretpostavka je da slični korisnici biraju slične proizvode i da entiteti koji odgovaraju jednom korisniku verovatno odgovaraju i drugom. 
<img src='assets/collaborative_filtering.png'>

U nastavku ćemo implementirati prototip sistema zasnovanog na uzajamnom filtriranju. 

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

Skup podataka sa kojim ćemo raditi zove se `MovieLense`. Kao što se iz imena naslućuje, ovaj skup sadrži informacije o filmovima, korisnicima i ocenama koje su korisnici dali filmovima. Skup se može preuzeti sa [zvanične adrese](https://grouplens.org/datasets/movielens/) u okviru sekcije `MovieLens 100K Dataset`. 

Fajl `data.csv` (preimenovani `u.data` fajl) sadrži informacije o ocenama filmova od strane korisnika. Svaki korisnik ima svoj jedinstveni identifikator i svaki film, takođe, ima svoj jedinstveni identifikator. Jedna vrsta ove datoteke sadrži identifikator korisnika, identifikator filma, ocenu na skali od 1 do 5 i vreme glasanja. Učitajmo skup podataka i prikažimo neke osnovne informacije o njemu.

Sada ćemo kreirati matricu čiji element na poziciji $(i, j)$ označava vrednost ocene korisnika $i$ za film $j$. Matrica će imati `number_of_users` redova i `number_of_items` kolona. Preostali elementi u matrici će biti postavljeni na 0.

Dalje ćemo kreirati matricu sličnosti korisnika. U pitanju je kvadratna matrice dimenzije `number_of_users` čije vrednosti predstavljaju kosinusnu sličnost između vektorskih reprezentacija korisnika. Vrednosti koje će se dobiti biće iz intervala $[0, 1]$. 

Neka $v_i$ i $v_j$ predstavljaju vektore ocena korisnika $i$ i $j$. Tada je element $(i, j)$ matrice sličnosti jednak 

$$\cos \angle(v_i, v_j) = \frac{v_i \cdot v_j}{||v_i||||v_j||},$$


Neka je $x_i$ vektor ocena korisnika $i$, $\overline{x}_i$ srednja vrednost ocena korisnika $i$, $x_{ij}$ ocena korisnika $i$ za film $j$ i $s_{ik}$ vrednost sličnosti korisnika $i$ i $k$. Za ocenu koju sistem generiše za korisnika $i$ i film $j$ zasnivaće se na Pirsonovom koeficijentu korelacije i biće oblika

$$x_{ij} = \overline{x}_i + \frac{\sum_{k \ne i}s_{ik}(x_{kj}-\overline{x}_k)}{\sum_{k \ne i}s_{ik}}.$$

Neki korisnici su strožiji i daju niske ocene svim filmovima, neki su pak blagonakloni pa daju više ocene svima. Smisao razmatranja prosečne vrednosti ocena korisnika je u cilju kreiranja sistema koji je objektivan. 

Možemo primetiti i da se ocena sistema računa uzimajući u obzir sve korisnike i njihove ocene. U praksi se ovaj pristup može ograničiti na razmatranje ocena nekoliko najsličnijih korisnika korisniku $x_i$. 

Sledeća funkcija na osnovu vrednosti matrice ocena $X$, matrice sličnosti $S$ između korisnika, indeksa korisnika $i$ i filma $j$, izračunava ocenu korisnika $i$ za film $j$.

In [2]:
def prediction(X, S, i, j):
    pass

Sistemi za preporuku se ocenjuju poređenjem vrednosti koje generiše sistem nad test skupom i tačnih vrednosti u test skupu. Skup `MovieLense` sadrži i 5 skupova za unakrsnu validaciju koji se mogu iskoristiti za ocenu sistema. Parametri sistema poput funkcije sličnosti između korisnika, broj korisnika koji se uzima u obzir i slično, mogu se birati tako da, na primer, minimizuju srednjekvadratnu grešku.

Rešenje za sistem preporuka koje je pobedilo na Netflix izazovu 2009. godine i osvojilo nagradu od 10 milona dolara bilo je bazirano na SVD dekompoziciji matrice ocena. 

Osnovna ideja korišćenja dekompozicija matrice u sistemima preporuka je učenje takozvanih ugnježdenih reprezentacija korisnika $U \in R^{mxd}$ i proizvoda $V \in R^{nxd}$ za zadatu matricu ocena $A \in R^{mxn}$. <img src='assets/svd_recommendation.png'>
Svaki red matrice $U$ predstavlje jednog od $m$ korisnika, a svaki red matrice $V$ jedan od $n$ proizvoda u ugnježdenom, latentnom, prostoru. Dimenzija ugnježdenog prostora $d$ je obično mnogo manja od vrednosti $n$ i $m$ pa je manipulacija matricama $U$ i $V$ efikasnija.  

Matrice $U$ i $V$ se biraju tako da proizvod $UV^T$ dobro aproksimira matricu $A$ tj. da minimizuju $||A-UV^T||_F^2$ Frobenijusovo rastojanje između matrice A i matrice $UV^T$.

Ovakve matrice se mogu dobiti SVD dekompozicijom. S obzirom da je u praksi matrica $A$ retka matrica i da je njena matrica $UV^T$ vrlo verovatno bliska nula matrici mogu se dobiti loše generalizacije. Jedna od popravki može biti  korišćenje faktorizacije sa težinama koja minimizuje funkciju $$\sum_{(i, j) \in obs }{(A_{ij} - <U_i, V_j>)^2} + w_0\sum_{(i, j)\notin obs}{(<U_i, V_j>)^2}$$ u kojoj $obs$ predstavljaju poznate ocene. Tako se može parametrom $w_0$ kontrolisati uticaj oba faktora. 

Dalje sledi jedan od referisanih načina da se iskosriti SVD dekompozicija u sistemima prepuruka.

Opredelićemo se za ugnježdavanje u prostor dimenzije d = 700. 

Nove matrice $U$ i $V$ koje faktorizuju matricu ocena će biti $U= U_{svd}\sqrt{S_d}$ i $VT = \sqrt{S_{d}}VT_{svd}$

SVD dekompozicija matrica preporuka, njena unapređenja i mnoge druge dekompozicije podržane su bibliotekom [Surprise](http://surpriselib.com/). Nju možemo instalirati komandom `conda install -c conda-forge scikit-surprise` u skladu sa [zvaničnim smernicama](https://anaconda.org/conda-forge/scikit-surprise). 

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import model_selection 
from surprise import accuracy

Ova biblioteka ima svoj skup funkcija za učitavanje i pripremu skupa podataka. `ml-100k` je ime MovieLense skupa koji smo koristili u prethodnom radu.

In [4]:
data = Dataset.load_builtin('ml-100k')

Dalji princip rada sa ovom bibliotekom odgovara standardnom `scikit-learn` protokolu. Podaci se dele na skup na treniranje i testiranje, biraju se i konfigurišu raspoloživi modeli, pozivom metode `fit` vrše se faktorizacije matrica, a pozivom metode `test` se dobijaju predikcije sistema. Ovaj postupak se iterativno ponavlja dok se ne odredi konfiguracija koja daje odgovarajuću tačnost. 

In [5]:
train_data, test_data = model_selection.train_test_split(data, test_size=.25)

Ova biblioteka SVD dekompoziciju izračunava korišćenjem optimizacionih tehnika (stohastičkog gradijentnog spusta) pa se navedeni parametri odnose na broj iteracija, korak učenja i regularizaciju. 

In [6]:
svd_algorithm = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

In [7]:
svd_algorithm.fit(train_data)

In [8]:
predictions = svd_algorithm.test(test_data)

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9667


0.9667056913436141

In [10]:
svd_algorithm.predict(2, 10)

Prediction(uid=2, iid=10, r_ui=None, est=3.5320133333333334, details={'was_impossible': False})